In [1]:
'''
This used to scrape indeed.com for infromation for a number of companies.

The list of companies should be provided in the companies.csv (./data directory), with no index column.
Example:
    RBC
    TD bank
    Boston Pizza 

The result will be placed in the ./data directory, in companies_with_indeed_data.csv
File format:
    'company','work_life','salary','job_security','management','culture','ceo','ceo_details',
            'num_reviews','pay_details', 'num_employees', 'revinew','got_indeed_review'
'''

"\nThis used to scrape indeed.com for infromation for a number of companies.\n\nThe list of companies should be provided in the companies.csv (./data directory), with no index column.\nExample:\n    RBC\n    TD bank\n    Boston Pizza \n\nThe result will be placed in the ./data directory, in companies_with_indeed_data.csv\nFile format:\n    'company','work_life','salary','job_security','management','culture','ceo','ceo_details',\n            'num_reviews','pay_details', 'num_employees', 'revinew','got_indeed_review'\n"

In [2]:
from bs4 import BeautifulSoup
from datetime import datetime
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
import pandas as pd
import requests
import selenium.webdriver as webdriver
import timeit

# Scrape indeed for ratings

In [3]:
'''
This function is finding company on indeed, scrapes, 
returns array with the info.
@company_name is the name of the company (ex: "RBC")
'''

def get_ratings(company_name):


    try:
        # search for company
        driver.get("https://ca.indeed.com/companies")
        driver.find_element_by_id("search-by-company-input").send_keys(company_name)
        driver.find_element_by_id("cmp-discovery-cs-submit").click()
        
        # if company found, open the company's page
        elements = driver.find_elements_by_css_selector("div.cmp-company-tile-blue-name a")
        main_url =elements[0].get_attribute("href")
        driver.get(main_url)
        
        # start return verctor with just name of the company
        all_ratings = [company_name]
        
        # find page elements
        ratings = driver.find_elements_by_class_name("cmp-ReviewCategory-rating")
        cnt = driver.find_elements_by_class_name("cmp-CeoWidgetWithRating-subtext")
        ceo_appr = driver.find_elements_by_class_name("cmp-CeoWidgetWithRating-percent")
        num_reviews = driver.find_elements_by_class_name("cmp-ReviewAndRatingsStory-reviewsLink")
        if (ratings):
            for r in ratings:
                all_ratings.append(r.text)
        else:
            all_ratings+=(['','','','',''])
        try:
            all_ratings.append(ceo_appr[0].text)
        except:
            all_ratings.append('')
        try:
            all_ratings.append(cnt[0].text)
        except:
            all_ratings.append('')
        try:
            all_ratings.append(num_reviews[0].text)
        except:
            all_ratings.append('')
        try:
            salaries_url = main_url+"/salaries"
            driver.get(salaries_url)
            num_reviews = driver.find_elements_by_class_name("cmp-salary-footer-text")
            all_ratings.append(num_reviews[0].text)
        except:
            all_ratings.append('')
        
        # collect additional data (company size and revenue)
        emp = '0'
        rev = '0'
        try:
            about_url = main_url+"/about"
            r = requests.get(about_url,
                         headers={
                             "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.12; rv:62.0) Gecko/20100101 Firefox/62.0"},
                         timeout=10)
            soup = BeautifulSoup(r.content, "html5")
            ddd = soup.find("dl", {"id": "cmp-company-details-sidebar"})
            d1_data = ddd.find_all("dd")
            d2_data = ddd.find_all("dt")

            for name, item in zip(d2_data,d1_data):
                if ('Employees' in name):
                    emp = item
                if ('Revenue' in name):
                    rev = item
        finally:
            all_ratings.append(emp) 
            all_ratings.append(rev) 
            
        all_ratings.append('1')
        return all_ratings
    except:
        return [company_name,'','','','','','','','','','','','0']


# Main



In [8]:

'''
Loads the list of the companies from csv file,
starts the driver, calls get_ratings() for each company,
creates and cleans DataFrame with the results,
saves results into companies_with_indeed_data.csv
'''

# read csv with just list of company names
df_companies = pd.read_csv("data/companies.csv")

start_time = timeit.default_timer()
# 1. start the driver
binary = FirefoxBinary('C:\\Program Files\\Mozilla Firefox\\firefox.exe')
driver = webdriver.Firefox(firefox_binary=binary)

start_time = timeit.default_timer()
result =[]
# df_companies = df_companies[:2]

for ii,i in df_companies.iterrows():
    m = get_ratings(i[0]) 
    result.append(m)

elapsed = timeit.default_timer() - start_time

# done with that driver
driver.close()

print ('Time spent scraping:', elapsed )

mt = pd.DataFrame(result,)
mt.columns=['company','work_life','salary','job_security','management','culture','ceo','ceo_details',
            'num_reviews','pay_details', 'num_employees', 'revinew','got_indeed_review']

#the following is a fack, to make sure all nulls a set to Nan
mt.to_csv("temp.csv", index=False)
mt = pd.read_csv("temp.csv")

# clean up data collected
mt['num_reviews'].fillna('0 ', inplace = True)
mt['num_reviews'] = [i.replace('See all ','') for i in mt['num_reviews']]
mt['num_reviews'] = [i.split()[0] for i in mt['num_reviews']]
mt['num_employees'].fillna("<dd></dd>", inplace = True)
mt['num_employees'] = [i.replace('<dd>','') for i in mt['num_employees']]
mt['num_employees'] = [i.replace('</dd>','') for i in mt['num_employees']]

mt.to_csv('data/companies_with_indeed_data.csv', index = False)


Time spent scraping: 65.92224540000001
